In [20]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc

# Read the Parquet file
file_path = "/Users/Macington/Documents/Projects/Project Gambit/Games/ply.parquet"
table = pq.read_table(file_path)
df = table.to_pandas()

print(df)


                      pgn_id  ply             board_sum     progression_hash
0        4966373121947802697    0  18446462598732906495  6548726006382385350
1        1613272150230853183    0  18446462598732906495  6548726006382385350
2        1613272150230853183    1  18446462598735003583  6838202551196677307
3         127762129624462438    0  18446462598732906495  6548726006382385350
4         127762129624462438    1  18446462598749683455 -1959233720066286525
...                      ...  ...                   ...                  ...
7054364 -2014757526705662466  236  13835102769989829121 -3085291962456116356
7054365 -2014757526705662466  237  13835102769985634817  2208772014170310088
7054366 -2014757526705662466  238  13835102767846539777  5917023950694882328
7054367 -2014757526705662466  239  13835102767578105345 -7270769053707336975
7054368 -2014757526705662466  240  13835102492700198401  7625980533342292681

[7054369 rows x 4 columns]


In [9]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# Function to compute a progressive hash for a group of board_sum values
def progressive_hash(group):
    hashes = []
    progression = []
    for value in group['board_sum']:
        progression.append(value)
        hashes.append(hash(tuple(progression)))
    group['progression_hash'] = hashes
    return group

# Read the Parquet file
file_path = "/Users/Macington/Documents/Projects/Project Gambit/Games/Ply.parquet"
table = pq.read_table(file_path)
df = table.to_pandas()

# Apply the progressive hash function to each group by 'game_id'
df = df.groupby('game_id').apply(progressive_hash)

# Convert the updated DataFrame back to a pyarrow Table
table_with_progression_hash = pa.Table.from_pandas(df)

# If you want to write the updated table back to a Parquet file
output_file_path = "/Users/Macington/Documents/Projects/Project Gambit/Games/Ply_with_progression_hash.parquet"
pq.write_table(table_with_progression_hash, output_file_path)

print(df)


                           game_id  ply             board_sum  \
game_id                                                         
1.844646e+19 0        1.844646e+19    0  18446462598732906495   
3.689293e+19 2705610  3.689293e+19    0  18446462598732906495   
             2705611  3.689293e+19    1  18446462598735003583   
             2705612  3.689293e+19    0  18446462598732906495   
             2705613  3.689293e+19    1  18446462598749683455   
...                            ...  ...                   ...   
3.317226e+21 2798774  3.317226e+21  236  13835102769989829121   
             2798775  3.317226e+21  237  13835102769985634817   
             2798776  3.317226e+21  238  13835102767846539777   
             2798777  3.317226e+21  239  13835102767578105345   
             2798778  3.317226e+21  240  13835102492700198401   

                         progression_hash  
game_id                                    
1.844646e+19 0        6548726006382385350  
3.689293e+19 2705610  